In [65]:
from dfs_service import DFS, distance_obj,filter_dups,find_best_connection_distance,MySolutionMap
from dfs_service import connection_exists,validate_connection, calculate_path_distance
from itertools import permutations
import settings
import time
from decimal import Decimal
from utils import load_maps
from collections import defaultdict

In [42]:
distances = distance_obj()
ride_map = load_maps()

In [93]:
ride_map[64430]['name']

"gadget's go coaster"

In [3]:
 # nested loop,region to region connection
all_regions_connections = permutations([region_name for region_list, region_name in settings.REGIONS_LIST])
all_regions_connections = list(all_regions_connections)
print(f" permutations of all regions {len(list(all_regions_connections))}\n")

 permutations of all regions 362880



In [4]:
# ride_map

In [10]:
solution_map = MySolutionMap()



t0 = time.time()
for region_list, region_name in (settings.REGIONS_LIST +     [(settings.FANTASY_LAND | settings.TOON_TOWN, "FANTASY_TOON")] ):
    t1 = time.time()
    d = DFS(region_list)
    solutions = d.start()
    filtered_solutions = filter_dups(solutions, distances)
    # solution_map[region_name] = d.start()
    solution_map[region_name] = filtered_solutions

    print(
        f" {region_name} region took {time.time() - t1}, total solutions {len(solutions)} after filter {len(filtered_solutions)}\n")

print(f" full searches took {time.time() - t0}")



            

 STAR_WARS_GALAXY_EDGE region took 0.022483110427856445, total solutions 2 after filter 2

 TOON_TOWN region took 0.03964996337890625, total solutions 1728 after filter 54

 CRITTER_COUNTRY region took 0.0020139217376708984, total solutions 6 after filter 6

 NEW_ORLEANS_SQUARE region took 0.0019469261169433594, total solutions 4 after filter 4

 FRONTIER_LAND region took 0.0020258426666259766, total solutions 4 after filter 4

 ADVENTURE_LAND region took 0.0018661022186279297, total solutions 4 after filter 4

 MAIN_STREET region took 0.01116800308227539, total solutions 648 after filter 42

 TOMORROW_LAND region took 0.0054950714111328125, total solutions 72 after filter 24

1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
 FANTASY_LAND region took 30.546197175979614, total solutions 237973 after filter 24

1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
 FANTASY_TOON region took 34.10088515281677, total solutions 14745

In [84]:
#making sure path exists backwards too
ttt = 0
ccc = 0

for r in solution_map:
    
    tmp = {}
    for k1,k2 in solution_map[r]:
        
        if (k2,k1) not in solution_map[r]:
            ttt+=1
#             solution_map[r][ (k2,k1)  ] = list(reversed(solution_map[r][( k1,k2)  ]))
            tmp[ (k2,k1)  ] = list(reversed(solution_map[r][( k1,k2)  ]))
    
        ccc +=1
    solution_map[r].update(tmp)
        
        
        

print(ttt, ccc)

20 174


In [ ]:
solution_map["NEW_ORLEANS_SQUARE"]

In [19]:
# permutation of all region connects
# we need to start in main street, so no reason to generate permutations where main street is not first
perms = permutations([name for _, name in settings.REGIONS_LIST if name not in { settings.MAIN_STREET_NAME, settings.TOON_TOWN_NAME}])
perms = list(perms)
print(len(perms))
print(f"all region names {[name for _, name in settings.REGIONS_LIST if name not in { settings.MAIN_STREET_NAME, settings.TOON_TOWN_NAME}]}")
perms = [[settings.MAIN_STREET_NAME] + list(p) for p in perms]
print(len(perms))

print(f"found {len(perms)} region to region paths")
print(f"path contains {len(perms[0]) } elements")

5040
all region names ['STAR_WARS_GALAXY_EDGE', 'CRITTER_COUNTRY', 'NEW_ORLEANS_SQUARE', 'FRONTIER_LAND', 'ADVENTURE_LAND', 'TOMORROW_LAND', 'FANTASY_LAND']
5040
found 5040 region to region paths
path contains 8 elements


In [18]:
perms2 = [name for _, name in settings.REGIONS_LIST if name not in { settings.MAIN_STREET_NAME, settings.TOON_TOWN_NAME, settings.FANTASY_LAND_NAME}]

perms2.append("FANTASY_TOON")

perms2 = permutations(perms2)

perms2 = [[settings.MAIN_STREET_NAME] + list(p) for p in perms2]

print(len(perms2))



5040


In [58]:
# for p in perms2:
#     if p.index("MAIN_STREET") != 0:
#         print(p)

In [ ]:

# print(settings.REGIONS_LIST)

magic = ['MAIN_STREET', 'ADVENTURE_LAND', "FRONTIER_LAND", "NEW_ORLEANS_SQUARE", "CRITTER_COUNTRY", "STAR_WARS_GALAXY_EDGE", "FANTASY_LAND"]
for idx, p in enumerate(perms):
    
    
    if (magic[0] == p[0]) and (magic[1] == p[1]) and (magic[2] == p[2]) and (magic[3] == p[3]):
        if (magic[4] == p[4]) and (magic[5] == p[5]) and (magic[6] == p[6]):
            print(p, idx)
        
        
        
    

In [ ]:
# starting in main street,
# inside each solution for source region,check exit node
# check if exit node has connection with entry node from next region
# if none found, exit path

best_distance = float("inf")
my_inf = Decimal(float("inf"))
deadend = set()
longest_path = float("-inf")
# print(len(perms))
# print(perms[0])
best_solutions = []

FLOW_SOLUTION = []
rolling_solutions = []

for pidx, path in enumerate(perms):
    
    CURR_FLOW = []
    
    curr_path_distance = Decimal("0.0")
#     tmp_distance = 0
    
    #all exists from main entrance area
    source_valid = set(   [ e2 for (e1,e2) in solution_map[path[0] ]]    )
    rolling_steps = [source_valid, ]
    for i in range(1, len(path)):
        r_source = path[i - 1]
        r_target = path[i]
        
        if (r_source, r_target) in deadend:
            curr_path_distance = my_inf
            
            break
        
        
#         possible_sources = [ e2 for (e1,e2) in solution_map[r_source] ]
        
        
        tmp_distance = find_best_connection_distance(r_source, r_target, solution_map)
        source_valid = validate_connection(r_source, r_target, solution_map, source_valid)
       
        if len(source_valid) == 0:
            print(f"no VALID connection between {r_source} {r_target}, {i}th in path {pidx}")
            deadend.add((r_source, r_target))
            break
        
        
        rolling_steps.append(source_valid)
        
        curr_path_distance += tmp_distance
        
        if tmp_distance == my_inf:
            print(f"no connection between {r_source} {r_target}, {i}th in path {pidx}")
            deadend.add((r_source, r_target))
            if i >= 7:
                print(f"{path}")
            
            print("\n")
            longest_path = max(longest_path, i)
#             curr_path_distance = float("inf")
            break
            
#         else:

        
    
    if  curr_path_distance != my_inf  and (i == len(path)-1):
#     if  curr_path_distance != my_inf:
        best_distance = min(best_distance, curr_path_distance)
        if curr_path_distance == best_distance:
            best_path = path
            
        best_solutions.append( list(path) )
        rolling_solutions.append( list(rolling_steps ) )
        
#     if pidx == 3295:
#         print( "@@@", i , len(path)-1  ,path, curr_path_distance )

print(f"best distance path: {best_distance}")
print(f"longest reached: {longest_path}")
print(f"scanned: {pidx}")
print(f"best solutions: {len(best_solutions)}")

In [ ]:
best_solutions

In [ ]:
rolling_solutions[0]

In [85]:
#try to go over all PERMS and run DFS at the same time
bad_jumps = set([])
best_tries = defaultdict(list)

def worm_search(curr_path_idx,  path, curr_ride_list, solutions_map, winners, distance_map):
    
    if curr_path_idx >= len(path):
        
        tot = calculate_path_distance(curr_ride_list,distance_map)
#         print(f"tot {tot}, curr_ride_list len {len(curr_ride_list)}")
        winners.append(  [tot, path, curr_ride_list]  )
        return
    
    
    #get list of (entrance, exist) nodes for current area
    for (e1, e2) in solutions_map[ path[curr_path_idx] ]:
        
        #eliminate if previous node,
        k1,k2 = e1,e2
        
        if distance_map.get( (curr_ride_list[-1], e1) ,-1) == -1:
            if  distance_map.get( (curr_ride_list[-1], e2) ,-1) == -1:
                bad_jumps.add( (path[curr_path_idx-1]  , path[curr_path_idx] , curr_ride_list[-1], e1,e2 )   )
                
                best_tries[curr_path_idx].append(  path )
                
                ###
                if e1 in {70528, 60282}  and e2 in {70528, 60282}:
                    print("WTF")
                
                ###
                continue
            
            else:
                k2,k1 = e1,e2
        
        
        
        worm_search(curr_path_idx+1,  path, curr_ride_list +solutions_map[ path[curr_path_idx] ][(k1,k2)]   , solutions_map, winners, distance_map)
        
        
        

winners = []
for pidx, path in enumerate(perms2):
    #path = [tomorrowland, mainentrance ...]
    
    for (e1,e2) in solution_map["MAIN_STREET"]:
        worm_search(  1, path, solution_map["MAIN_STREET"][(e1,e2)],solution_map,winners, distances )


# print(bad_jumps)
    

In [88]:
best_tries_printed = set()
for k in best_tries:
    print(k)
    if k >=7:
        for p in best_tries[k]:
            tt = tuple(p)
            if tt in best_tries_printed:
                continue
                
            best_tries_printed.add( tt )
            print(p)

1
6
7
['MAIN_STREET', 'FRONTIER_LAND', 'STAR_WARS_GALAXY_EDGE', 'CRITTER_COUNTRY', 'NEW_ORLEANS_SQUARE', 'ADVENTURE_LAND', 'TOMORROW_LAND', 'FANTASY_TOON']
['MAIN_STREET', 'FRONTIER_LAND', 'STAR_WARS_GALAXY_EDGE', 'CRITTER_COUNTRY', 'NEW_ORLEANS_SQUARE', 'ADVENTURE_LAND', 'FANTASY_TOON', 'TOMORROW_LAND']
4
5
3
2


In [94]:



for p in bad_jumps:
    if "FANTASY_TOON" == p[1]  and "TOMORROW_LAND" in p:
        #astro
        if 44978 in p:
            ## ride_map[64430]['name']
            print("$$$", [ride_map[xx]['name'] if xx in ride_map else xx  for xx in p ])
            
        else:
#             print(p)
            print([ride_map[xx]['name'] if xx in ride_map else xx  for xx in p ])



# should work:
# ('FRONTIER_LAND', 'FANTASY_TOON')
#('FANTASY_TOON', 'ADVENTURE_LAND')
#('STAR_WARS_GALAXY_EDGE', 'FANTASY_TOON')
#('FANTASY_TOON', 'MAIN_STREET')
#('FANTASY_TOON', 'TOMORROW_LAND')


['TOMORROW_LAND', 'FANTASY_TOON', 'autopia', 'sleeping beauty castle walkthrough', 'rail road mickey town']
['TOMORROW_LAND', 'FANTASY_TOON', 'autopia', 'sleeping beauty castle walkthrough', "gadget's go coaster"]
['TOMORROW_LAND', 'FANTASY_TOON', 'autopia', 'sleeping beauty castle walkthrough', "goofy's playhouse"]
['TOMORROW_LAND', 'FANTASY_TOON', 'autopia', 'rail road mickey town', 'sleeping beauty castle walkthrough']
['TOMORROW_LAND', 'FANTASY_TOON', 'autopia', "donald's boat", 'sleeping beauty castle walkthrough']
['TOMORROW_LAND', 'FANTASY_TOON', 'finding nemo submarine voyage', 'rail road mickey town', 'sleeping beauty castle walkthrough']
['TOMORROW_LAND', 'FANTASY_TOON', 'monorail', "mickey's house and meet mickey", 'sleeping beauty castle walkthrough']
['TOMORROW_LAND', 'FANTASY_TOON', 'rail road - tomorrow land', 'sleeping beauty castle walkthrough', "minnie's house"]
['TOMORROW_LAND', 'FANTASY_TOON', 'autopia', "chip 'n dale treehouse", 'sleeping beauty castle walkthrough'

In [90]:
#70528 sleeping beauty castle
#39868 roger rabbit
#99677 mickey house
#40788 minnie
#64430 gadget coaster
#60515 chip and dale tree house
#27887 donald boat
#1924 goofy house
#73844 rail road mickey town ****
#30923 small world

for k in solution_map["FANTASY_TOON"]:
    print(k)

(70528, 39868)
(70528, 99677)
(70528, 40788)
(70528, 64430)
(70528, 60515)
(70528, 27887)
(70528, 1924)
(70528, 73844)
(70528, 30923)
(39868, 70528)
(99677, 70528)
(40788, 70528)
(64430, 70528)
(60515, 70528)
(27887, 70528)
(1924, 70528)
(73844, 70528)
(30923, 70528)


In [70]:
#50720 disney gallery
#91073 rail road entrance
#70282 main street vehicles
#82766 lincoln
#83634 pumpkin
#40918 cinema
#60282 fortune teller mainstreet



myset= set()
for k in solution_map["MAIN_STREET"]:
    print(k)

# print(myset)

(50720, 60282)
(50720, 40918)
(50720, 83634)
(50720, 82766)
(50720, 70282)
(50720, 91073)
(91073, 60282)
(91073, 40918)
(91073, 83634)
(91073, 82766)
(91073, 70282)
(91073, 50720)
(70282, 60282)
(70282, 40918)
(70282, 83634)
(70282, 91073)
(70282, 82766)
(70282, 50720)
(82766, 60282)
(82766, 40918)
(82766, 83634)
(82766, 91073)
(82766, 70282)
(82766, 50720)
(83634, 60282)
(83634, 82766)
(83634, 40918)
(83634, 70282)
(83634, 91073)
(83634, 50720)
(40918, 60282)
(40918, 82766)
(40918, 83634)
(40918, 70282)
(40918, 91073)
(40918, 50720)
(60282, 40918)
(60282, 83634)
(60282, 91073)
(60282, 82766)
(60282, 50720)
(60282, 70282)


In [91]:
#68863 buzz
#23238 nemo
#46507 autopia
#44978   astro orbitor
#8115  monorail
#45812 space mountain
#58559 rail road tomorrow




myset= set()
for k in solution_map["TOMORROW_LAND"]:
    myset.add(k[0])
    myset.add(k[1])

print(myset)

{68863, 23238, 46507, 44978, 8115, 45812, 58559}


In [64]:
print(len(winners))
print(winners[0][0])
winners.sort(key=lambda x: x[0])
# # distances
print(winners[0][1])

print(winners[-1][0])
print(winners[-1][1])


test = {  tuple(p[1])  for p in winners  }
print(len(test))


0


IndexError: list index out of range

In [ ]:
solution_map['TOON_TOWN']

In [ ]:
### use DFS to find all combinations

all_possibilities = []

#path:   main street, tomorrowland, fantasyland ....
for pidx, path in enumerate(perms):
    
        path_possibilities = [  solution_map[path[0] ]  ]
        
        for i in range(1, len(path)):
            r_source = path[i - 1]
            r_target = path[i]
            #source, target = main street, tomorrowland
            
            #is it a dead end?
            ###
            if not connection_exists(r_source, r_target, solution_map):
                continue
            
            
            
            
            #get connections:
            #source = [  [entrance, store1 ], [entrance, store2]   ]
            #target = [  [orbital, space mountain], [ buzz, nemo]   ]
            
            path_possibilities.append( solution_map[r_target] )
        
        
        if len(path) == len(path_possibilities):
            all_possibilities.append( path_possibilities  )
        
            







In [ ]:
# all_possibilities[0]
# print(len(all_possibilities))
# print(len(all_possibilities[0]))
# print((all_possibilities[0]))

In [ ]:
#func

def end_end_path_build(curr_path, curr_idx, path_perms ):
    
    if curr_idx >= len(curr_path):
        return True
    
    

#pick first index, check if 

In [ ]:
# for posib in all_possibilities:
    
    

In [ ]:
print(deadend)

In [ ]:
print(len(best_solutions))
print(best_path)

In [ ]:

conns = (
#     ("CRITTER_COUNTRY", "NEW_ORLEANS_SQUARE"),
#     ("ADVENTURE_LAND", "NEW_ORLEANS_SQUARE"),
#     ("ADVENTURE_LAND", "FRONTIER_LAND"),
#      ("NEW_ORLEANS_SQUARE", "FRONTIER_LAND"),
#      ("NEW_ORLEANS_SQUARE", "ADVENTURE_LAND"),
#          ("NEW_ORLEANS_SQUARE", "CRITTER_COUNTRY"),
#              ("MAIN_STREET", "ADVENTURE_LAND"),
#     ("ADVENTURE_LAND", "FRONTIER_LAND"),
#     ("FRONTIER_LAND", "NEW_ORLEANS_SQUARE"),
#     ("NEW_ORLEANS_SQUARE","CRITTER_COUNTRY",),
#     ("CRITTER_COUNTRY", "STAR_WARS_GALAXY_EDGE"),
#     ( "STAR_WARS_GALAXY_EDGE", "FANTASY_LAND"),
#     ("FANTASY_LAND", "TOMORROW_LAND"),
    
#         ("NEW_ORLEANS_SQUARE", "STAR_WARS_GALAXY_EDGE"),
#                  ("MAIN_STREET", "NEW_ORLEANS_SQUARE"),
    
    ("MAIN_STREET", "TOMORROW_LAND"),
    ("FANTASY_LAND", "TOMORROW_LAND"),
    ("FANTASY_LAND", "ADVENTURE_LAND"),
    ("FRONTIER_LAND", "ADVENTURE_LAND"),
    ("FRONTIER_LAND", "CRITTER_COUNTRY"),
    ("NEW_ORLEANS_SQUARE", "CRITTER_COUNTRY"),
    ("NEW_ORLEANS_SQUARE", "STAR_WARS_GALAXY_EDGE"),
    ("FANTASY_LAND", "TOON_TOWN"),


    
    
    


)

for r_source, r_target in conns:
    tmp_distance = find_best_connection_distance(r_source, r_target, solution_map)
    print(r_source, r_target,  tmp_distance, (r_source, r_target) in deadend )

In [ ]:
def find_best_toon_town_fantasy_link(solution_map, distances):
    # check each end node from region1 against all start node from region2
#     tmp_distance = Decimal(float("inf"))
    # get all solutions for source
    best_combo = None
#     resp_source, resp_target = None, None
    resp = Decimal("inf")
#     for sol_source in solution_map["FANTASY_LAND"]:
    for  f1  in settings.FANTASY_LAND: 
        for sol_target in solution_map["TOON_TOWN"]:
#             s1, s2 = sol_source[0], sol_source[-1]
#             t1, t2 = sol_target[0], sol_target[-1]
            t1,t2 = sol_target[0], sol_target[-1]

#             tmp_distance = min(distances.get((s1, t1), float("inf")), tmp_distance)
#             tmp_distance = min(distances.get((s1, t2), float("inf")), tmp_distance)
#             tmp_distance = min(distances.get((s2, t1), float("inf")), tmp_distance)
#             tmp_distance = min(distances.get((s2, t2), float("inf")), tmp_distance)
            tmp_distance1 = distances.get((f1, t1), Decimal("inf"))
                
            if tmp_distance1 == Decimal("inf"):
                continue
            
            tmp_distance2 = calculate_path_distance( solution_map["TOON_TOWN"][(t1,t2)]  ,distances )
            buff = Decimal('inf')
            f2_best = None
            buff = Decimal('inf')
            for f2 in  settings.FANTASY_LAND: 
                if f1 != f2:
                    continue
                    
                tmp = distances.get( (t2, f2) , Decimal("inf"))
                buff = min(buff, tmp)
                if buff == tmp:
                    f2_best = f2
            
            if buff != Decimal("inf"):
                print(tmp_distance1,tmp_distance2,buff)
                
            tmp_distance = tmp_distance1+tmp_distance2+buff
            
            resp = min(resp, tmp_distance)
            
            if resp == tmp_distance:
                best_combo = (  f1, t1,t2,f2_best )

    return (resp, best_combo)


z = find_best_toon_town_fantasy_link(solution_map, distances)

In [ ]:
z

In [ ]:
for key in solution_map['FANTASY_LAND']:
    
    k1, k2 = key
    k1,k2 = ride_map[k1]['name'],ride_map[k2]['name']
    
    print(f"{(k1,k2)}: {[   ride_map[e]['name']  for e in solution_map['FANTASY_LAND'][key]    ]}\n\n")


In [ ]:
for key in solution_map['ADVENTURE_LAND']:
    
    k1, k2 = key
    k1,k2 = ride_map[k1]['name'],ride_map[k2]['name']
    
    print(f"{(k1,k2)}: {[   ride_map[e]['name']  for e in solution_map['ADVENTURE_LAND'][key]    ]}\n\n")